# 🤖 Conversational State Machine with LangGraph

In this notebook, we'll build a **conversational state machine** using LangGraph. This will allow an LLM-powered agent to handle branching logic and state tracking across multiple user turns.


## 📦 Install Dependencies

Install LangGraph and LangChain if you haven't already:


In [24]:
!pip install langgraph langchain openai


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## ⚙️ Setup and Imports

We’ll use OpenAI for the LLM and define a simple state machine with three nodes:

- `classify_node`: to determine intent
- `answer_node`: to respond to questions
- `exit_node`: to end the session


In [25]:
from langchain.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, END
from typing import TypedDict, Literal
from langchain_core.runnables import RunnableLambda
import os

# Make sure your OpenAI key is set
#os.environ["OPENAI_API_KEY"] = "sk-..."

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## 🧠 Define State Schema

LangGraph passes data between nodes in a dictionary. We'll define the input/output shape.


In [26]:
class ChatState(TypedDict):
    input: str
    classification: Literal["question", "exit", "unknown"]
    output: str

## 🧩 Define Graph Nodes (Functions)


In [27]:
def classify_node(state):
    user_input = state["input"]
    prompt = f"Classify the intent of the user message: '{user_input}'. Reply with 'question', 'exit', or 'unknown'."
    result = llm.invoke(prompt).content.lower()
    classification = "question" if "question" in result else "exit" if "exit" in result else "unknown"
    return {**state, "classification": classification}

def answer_node(state):
    user_input = state["input"]
    result = llm.invoke(f"Answer the user's question: {user_input}").content
    return {**state, "output": result}

def exit_node(state):
    return {**state, "output": "Goodbye! Ending the conversation."}

## 🔁 Build and Compile the State Graph


In [28]:
# Initialize the graph
graph = StateGraph(ChatState)

# Add nodes
graph.add_node("classify", classify_node)
graph.add_node("answer", answer_node)
graph.add_node("exit", exit_node)

# Set the entry point
graph.set_entry_point("classify")

# Define the routing function
def route(state: ChatState) -> str:
    return state["classification"]

# Add conditional edges
graph.add_conditional_edges(
    source="classify",
    path=route,
    path_map={
        "question": "answer",
        "exit": "exit",
        "unknown": "exit"
    }
)

# Add edges to END
graph.add_edge("answer", END)
graph.add_edge("exit", END)

# Compile the graph
app = graph.compile()

## 💬 Run the State Machine


In [30]:
print("🧠 LangGraph Chat is ready. Type 'exit' to quit.\n")

while True:
    user_input = input("👤 You: ")

    # Exit manually
    if user_input.lower().strip() in {"exit", "quit"}:
        print("🤖 Agent: Goodbye!")
        break

    state = {"input": user_input}
    response = app.invoke(state)

    print("🤖 Agent:", response["output"])

    # Exit automatically if agent classifies it as exit
    if response.get("classification") == "exit":
        break


🧠 LangGraph Chat is ready. Type 'exit' to quit.

🤖 Agent: The capital of France is Paris.
🤖 Agent: Goodbye!
